In [2]:
import numpy as np
from Helpers import *
from mainSVT import *
import time
import random
import scipy.sparse as sp

In [3]:

# Set random seed for reproducibility
np.random.seed(42)

# Load the data from the specified .npz file
file = np.load(".\\data\\2024-03-29-data.npz")

# Extract the necessary components from the file
data = file['data']               # Raw Data
mz_values = file['mz_values']     # Corresponding mz-values
image_size = file['image_size']   # Size of the image
mz_picks = file['mz_picks']       # Selected mz-ids
num = file['num']                 # Number of picks


In [6]:
def svt(n_data_sampled, step_size, tolerance, tau, max_iter, max_rank=10):
    
    # Initialize k and a Y_0
    k = 1
    Y_0 = k_0_finder(tau, step_size, n_data_sampled) * step_size * n_data_sampled
    
    # First singular value calculation
    u, s, v = svd(Y_0, full_matrices=False)
    print('The biggest singular value on iteration 0:' , s[0])
    
    
    # Get list ready
    iterate = True
    
    rank_arr = []
    rel_error_s_arr = []

    while iterate:
        if k == 1:
            Y = Y_0

        # Singular Value Decomposition
        U, sigma, Vt = svd(Y, full_matrices=False)
        
        # Thresholding
        big_sigma = sigma[0]
        print(f'Biggest singular value = {big_sigma}')
        sigma_thresh = np.maximum(sigma - tau, 0)
        # Check for rank stopping condition
        rank = np.sum(sigma_thresh > 0)
        # Construct the approximation matrix X
        X = U[:, :rank] @ np.diag(sigma_thresh[:rank]) @ Vt[:rank, :]

        # Calculate errors
        error_sampled_matrix = -projection_operator(M_sampled=n_data_sampled, X=X)
        rel_error_s = (norm(projection_operator(n_data_sampled, X=X), 'fro') / norm(n_data_sampled, 'fro')) 
        
        if rel_error_s<1e-2:
            step_size *=.9
        
        # Update Y
        Y = Y + step_size * error_sampled_matrix
        
        # Append results for plotting
        rank_arr.append(rank)
        rel_error_s_arr.append(rel_error_s)
        
        # Check stopping conditions
        if rank > max_rank:
            break
        if suggested_stop(X, n_data_sampled, tolerance):
            break
        if k >= max_iter:
            break
        k += 1        
        print(f'Iter {k}; Relative error:', rel_error_s)
        print('rank:',rank)
        print('\n')
        
    return X, rank_arr, k 

In [7]:

def massbinImageComparisonGPT(data, sampled_data, reconstructed_data, colormap='cubehelix',massbin_of_interest=random.randint(1,499)):
    mz_value = mz_values[massbin_of_interest]
    
    colormaps = [colormap]*3
    
    data_matrix = data[:, massbin_of_interest].reshape((500, 100))
    sampled_data_matrix = sampled_data[:, massbin_of_interest].reshape((500, 100))
    reconstructed_data_matrix = reconstructed_data[:, massbin_of_interest].reshape((500, 100))
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))  # Create a figure with 1 row and 3 columns
    
    # Plot each matrix with its respective colormap
    min_val = np.min(data_matrix)
    max_val = np.max(data_matrix)
    for ax, matrix, cmap, title in zip(axes, [data_matrix, reconstructed_data_matrix,sampled_data_matrix], colormaps, ['Original Data', 'Reconstructed Data', 'Sampled Data']):
        cax = ax.imshow(matrix, aspect='equal', cmap=cmap, vmin=min_val, vmax=max_val)  # Set aspect to 'equal' and use same limits as original data
        ax.set_title(f'{title} Colormap')
        fig.colorbar(cax, ax=ax)
    
    # Add overall title and show the plot
    fig.suptitle(f'Side by Side Comparison of Massbin {massbin_of_interest}: {mz_value}', fontsize=16)
    plt.show() 
    
    
    
    
def calculate_sparsity(matrix):
    """
    Calculate the sparsity of a matrix.

    Parameters:
    matrix (numpy.ndarray or scipy.sparse.spmatrix): The matrix.

    Returns:
    float: The sparsity percentage of the matrix.
    """
    # Convert numpy matrix to sparse matrix if necessary
    if isinstance(matrix, np.ndarray):
        matrix = sp.csr_matrix(matrix)

    # Calculate total elements, zero elements, and sparsity
    total_elements = matrix.shape[0] * matrix.shape[1]
    zero_elements = total_elements - matrix.nnz
    sparsity = (zero_elements / total_elements) * 100
    
    return sparsity



def normalizeData(data):
    # Normalize the data
    row_max = data.max(axis=1)
    n_data = data / row_max[:, np.newaxis]
    n_data = n_data.T
    return n_data, row_max

def denormalizeData(n_data, row_max):
    # Denormalize the data
    denormalized_data = n_data.T * row_max[:, np.newaxis]
    return denormalized_data




def run_svt_and_compute_error(data, keep_ratio, step_size, tolerance, tau, max_iter, max_rank):
    start = time.time()
    data_sampled, coords, coords_TF, M_sampled_nans = sample_from_matrix_ratio(data, ratio_to_keep=keep_ratio, seed=42)
    
    n_data_sampled, row_max = normalizeData(data_sampled)
    
    X_n_opt, rank_arr, iters = svt(n_data_sampled, step_size, tolerance, tau, max_iter, max_rank)
    
    X_reconstructed = denormalizeData(X_n_opt, row_max)
    
    
    
    total_time = time.time()-start
    
    U, sigma, Vt = svd(X_reconstructed, full_matrices=False)
    
    sparsity = calculate_sparsity(Vt)
    
    g_error = general_error_relative(data, X_reconstructed)
    is_error = insampling_error_relative(data_sampled, X_reconstructed, coords_TF)
    os_error = out_of_sample_error_relative(data, X_reconstructed, coords_TF)
    
    return g_error, is_error, os_error, total_time, iters, sparsity, X_n_opt, X_reconstructed

def average_general_error(max_rank, keep_ratio,num_runs=5, tau=8*np.sqrt(max(n_rows, n_columns))):
    step_size = 1.2
    tolerance = 1e-4
    max_iter = 1000
    
    
    g_errors = []
    is_errors = []
    os_errors = []
    times = []
    iters_l = []
    spars_l = []
    for _ in range(num_runs):
        print('\n')
        print('\n')
        print('\n')
        print('RUNNING',_)
        print('\n')
        print('\n')
        print('\n')
        g_error, is_error, os_error, total_time, iters, sparsity, X_n_opt, X_reconstructed = run_svt_and_compute_error(data, keep_ratio, step_size, tolerance, tau, max_iter,max_rank)
        
        g_errors.append(g_error)
        is_errors.append(is_error)
        os_errors.append(os_error)
        times.append(total_time)
        iters_l.append(iters)
        spars_l.append(sparsity)
    
    avg_g_error = np.mean(g_errors)
    avg_is_error = np.mean(is_errors)
    avg_os_error = np.mean(os_errors)
    avg_time = np.mean(times)
    avg_iters = np.mean(iters_l)
    avg_spars = np.mean(spars_l)
    
    
    print('\n \n')
    print(f"At rank {max_rank} and keep ratio {keep_ratio}:")
    print(f"Average General Error: {100*avg_g_error}%")
    print(f"Average In-Sample Error: {100*avg_is_error}%")
    print(f"Average Out-of-Sample Error: {100*avg_os_error}%")
    print(f"Average Sparsity: {avg_spars}%")
    print(f"Average Time: {avg_time:.2f} seconds")
    print(f"Average Iterations: {avg_iters:.2f}")
    print("\n")

    
    return avg_g_error, avg_is_error, avg_os_error, avg_time, avg_iters,X_n_opt, X_reconstructed



In [8]:
# Calculate the average general error over five runs

g_,is_,os_,t,k,X_n_opt, X_reconstructed = average_general_error(max_rank=50, keep_ratio = 0.5,num_runs=1)







RUNNING 0






The biggest singular value on iteration 0: 1915.4563531424117
Biggest singular value = 1915.4563531424117
Iter 2; Relative error: 0.9107370725065953
rank: 1


Biggest singular value = 2478.230647118291
Iter 3; Relative error: 0.5756801663049503
rank: 1


Biggest singular value = 2704.6561836655287
Iter 4; Relative error: 0.5002293276308019
rank: 1


Biggest singular value = 2796.3745352922997
Iter 5; Relative error: 0.4861545146290675
rank: 1


Biggest singular value = 2834.0049757815195
Iter 6; Relative error: 0.4833202171238044
rank: 1


Biggest singular value = 2849.76569475497
Iter 7; Relative error: 0.48254117179154793
rank: 1


Biggest singular value = 2856.58787847525
Iter 8; Relative error: 0.444684484690668
rank: 2


Biggest singular value = 2858.2093754918733
Iter 9; Relative error: 0.40199789693772037
rank: 2


Biggest singular value = 2856.6584424565717
Iter 10; Relative error: 0.393045807064578
rank: 2


Biggest singular value = 2855.126276774532
Iter

Biggest singular value = 2849.050091775344
Iter 85; Relative error: 0.19649847617452018
rank: 35


Biggest singular value = 2848.955766958345
Iter 86; Relative error: 0.1948285001887461
rank: 36


Biggest singular value = 2848.856032656724
Iter 87; Relative error: 0.19418891636378707
rank: 36


Biggest singular value = 2848.794367006429
Iter 88; Relative error: 0.19356313973891018
rank: 38


Biggest singular value = 2848.76846729028
Iter 89; Relative error: 0.19079835095612305
rank: 39


Biggest singular value = 2848.673411770908
Iter 90; Relative error: 0.18918599721470722
rank: 40


Biggest singular value = 2848.586463780473
Iter 91; Relative error: 0.18693062675213967
rank: 42


Biggest singular value = 2848.49140475602
Iter 92; Relative error: 0.18538501899330256
rank: 43


Biggest singular value = 2848.410922276283
Iter 93; Relative error: 0.18306025641660553
rank: 45


Biggest singular value = 2848.3140601320156
Iter 94; Relative error: 0.17992586330073515
rank: 47


Biggest sing

In [10]:
g_,is_,os_,t,k = average_general_error(max_rank=10, keep_ratio = 0.5,num_runs=5)







RUNNING 0






The biggest singular value on iteration 0: 1909.8783908700798
Biggest singular value = 1909.8783908700798
Iter 2; Relative error: 0.9144030853803323
rank: 1


Biggest singular value = 2474.150137269323
Iter 3; Relative error: 0.5768065000771823
rank: 1


Biggest singular value = 2701.26956586554
Iter 4; Relative error: 0.5005938990773064
rank: 1


Biggest singular value = 2793.297540672029
Iter 5; Relative error: 0.4863663559693589
rank: 1


Biggest singular value = 2831.0617762688826
Iter 6; Relative error: 0.48350583976826406
rank: 1


Biggest singular value = 2846.8783587063785
Iter 7; Relative error: 0.4827225694401112
rank: 1


Biggest singular value = 2853.722788089168
Iter 8; Relative error: 0.4436185582000335
rank: 2


Biggest singular value = 2855.3128383973526
Iter 9; Relative error: 0.40124515878279526
rank: 2


Biggest singular value = 2853.7725286931877
Iter 10; Relative error: 0.39234765398563753
rank: 2


Biggest singular value = 2852.2537942489316


Biggest singular value = 2849.41045300011
Iter 33; Relative error: 0.26875978383912036
rank: 8


Biggest singular value = 2849.335775331945
Iter 34; Relative error: 0.2679355477878493
rank: 8


Biggest singular value = 2849.3032035279925
Iter 35; Relative error: 0.26770990011967344
rank: 8


Biggest singular value = 2849.30920461989
Iter 36; Relative error: 0.26758731713739015
rank: 8


Biggest singular value = 2849.3375200800865
Iter 37; Relative error: 0.26748922558064636
rank: 8


Biggest singular value = 2849.3764076383154
Iter 38; Relative error: 0.2674020712964021
rank: 8


Biggest singular value = 2849.4193624932304
Iter 39; Relative error: 0.26583118142706985
rank: 9


Biggest singular value = 2849.4471982465
Iter 40; Relative error: 0.2614000027994075
rank: 9


Biggest singular value = 2849.4163169043622
Iter 41; Relative error: 0.2606169481918012
rank: 9


Biggest singular value = 2849.406787749641
Iter 42; Relative error: 0.2604241601442282
rank: 9


Biggest singular value =

Iter 14; Relative error: 0.34632005716698944
rank: 3


Biggest singular value = 2846.2204352358017
Iter 15; Relative error: 0.3446665991465862
rank: 3


Biggest singular value = 2846.0989507874856
Iter 16; Relative error: 0.3442322761176015
rank: 3


Biggest singular value = 2846.070625273927
Iter 17; Relative error: 0.344026748229078
rank: 3


Biggest singular value = 2846.082940740895
Iter 18; Relative error: 0.3438822927050935
rank: 3


Biggest singular value = 2846.109334610952
Iter 19; Relative error: 0.3437662025204659
rank: 3


Biggest singular value = 2846.1380733447736
Iter 20; Relative error: 0.32686744081253855
rank: 4


Biggest singular value = 2845.9488484040917
Iter 21; Relative error: 0.3226492737518987
rank: 4


Biggest singular value = 2845.7813075303934
Iter 22; Relative error: 0.32179156834068096
rank: 4


Biggest singular value = 2845.684134862039
Iter 23; Relative error: 0.319019045837727
rank: 5


Biggest singular value = 2845.629893909714
Iter 24; Relative error:

Biggest singular value = 2850.0708288229803
Iter 47; Relative error: 0.2540312983674687
rank: 10


Biggest singular value = 2850.082823852647
Iter 48; Relative error: 0.2539550078060196
rank: 10


Biggest singular value = 2850.1082700235474
Iter 49; Relative error: 0.2538927152541079
rank: 10


Biggest singular value = 2850.140249671018
Iter 50; Relative error: 0.2538364546483684
rank: 10


Biggest singular value = 2850.1748140576647
Iter 51; Relative error: 0.2537844635781811
rank: 10


Biggest singular value = 2850.209962792554

 

At rank 10 and keep ratio 0.5:
Average General Error: 16.32234583000903%
Average In-Sample Error: 15.95303219246545%
Average Out-of-Sample Error: 16.683510267087236%
Average Sparsity: 0.0%
Average Time: 217.22 seconds
Average Iterations: 51.20




In [11]:
g_,is_,os_,t,k = average_general_error(max_rank=25, keep_ratio = 0.3,num_runs=5)







RUNNING 0






The biggest singular value on iteration 0: 1994.556334877544
Biggest singular value = 1994.556334877544
Iter 2; Relative error: 0.8614511586268988
rank: 1


Biggest singular value = 2319.0470998190576
Iter 3; Relative error: 0.6656284329155728
rank: 1


Biggest singular value = 2526.274066899792
Iter 4; Relative error: 0.5663793621589919
rank: 1


Biggest singular value = 2658.897806188356
Iter 5; Relative error: 0.5199037599084241
rank: 1


Biggest singular value = 2744.0427527012766
Iter 6; Relative error: 0.4991020942501749
rank: 1


Biggest singular value = 2798.920641237954
Iter 7; Relative error: 0.4898593007927554
rank: 1


Biggest singular value = 2834.457110466156
Iter 8; Relative error: 0.4856499563235953
rank: 1


Biggest singular value = 2857.5977200151297
Iter 9; Relative error: 0.4836315669229945
rank: 1


Biggest singular value = 2872.766926914887
Iter 10; Relative error: 0.4825900285687688
rank: 1


Biggest singular value = 2882.790008116561
Iter 1

Biggest singular value = 2890.156807247322
Iter 85; Relative error: 0.23957387881256093
rank: 14


Biggest singular value = 2889.9783004409715
Iter 86; Relative error: 0.2374952775722872
rank: 15


Biggest singular value = 2889.780698746403
Iter 87; Relative error: 0.2359892128462646
rank: 15


Biggest singular value = 2889.5960791031825
Iter 88; Relative error: 0.23424939469819642
rank: 16


Biggest singular value = 2889.4085997678435
Iter 89; Relative error: 0.23289208399559166
rank: 16


Biggest singular value = 2889.2250596314834
Iter 90; Relative error: 0.2322823553149279
rank: 16


Biggest singular value = 2889.083452385495
Iter 91; Relative error: 0.2313950342173217
rank: 17


Biggest singular value = 2888.9609964219403
Iter 92; Relative error: 0.23020715672154424
rank: 17


Biggest singular value = 2888.826901248599
Iter 93; Relative error: 0.22894980101547596
rank: 18


Biggest singular value = 2888.6812511378575
Iter 94; Relative error: 0.22717096322909508
rank: 19


Biggest 

Iter 67; Relative error: 0.2558626640357567
rank: 10


Biggest singular value = 2891.5452170508665
Iter 68; Relative error: 0.25486436354304487
rank: 10


Biggest singular value = 2891.521251983954
Iter 69; Relative error: 0.25439573420335393
rank: 10


Biggest singular value = 2891.5091171589543
Iter 70; Relative error: 0.25414493437989594
rank: 10


Biggest singular value = 2891.512561740307
Iter 71; Relative error: 0.25398555677683243
rank: 10


Biggest singular value = 2891.5306292687133
Iter 72; Relative error: 0.25386622863744773
rank: 10


Biggest singular value = 2891.560684406597
Iter 73; Relative error: 0.2537658963118009
rank: 10


Biggest singular value = 2891.599824513015
Iter 74; Relative error: 0.2536757853248345
rank: 10


Biggest singular value = 2891.6454626043956
Iter 75; Relative error: 0.253592118958987
rank: 10


Biggest singular value = 2891.6955064029275
Iter 76; Relative error: 0.2519305785449418
rank: 11


Biggest singular value = 2891.701410642171
Iter 77; Re

Biggest singular value = 2890.153709668296
Iter 48; Relative error: 0.27427971859924855
rank: 8


Biggest singular value = 2890.114826676828
Iter 49; Relative error: 0.27050662269573356
rank: 8


Biggest singular value = 2890.0237321240566
Iter 50; Relative error: 0.2688977894650549
rank: 8


Biggest singular value = 2889.939700352662
Iter 51; Relative error: 0.2681541284420149
rank: 8


Biggest singular value = 2889.8838817190276
Iter 52; Relative error: 0.2677591441242751
rank: 8


Biggest singular value = 2889.859784876321
Iter 53; Relative error: 0.26750852769721745
rank: 8


Biggest singular value = 2889.8637287069837
Iter 54; Relative error: 0.2673210765280782
rank: 8


Biggest singular value = 2889.8898748167553
Iter 55; Relative error: 0.2671640926492793
rank: 8


Biggest singular value = 2889.9324479644924
Iter 56; Relative error: 0.26702408158959484
rank: 8


Biggest singular value = 2889.986562606625
Iter 57; Relative error: 0.2668952238780872
rank: 8


Biggest singular valu

Iter 28; Relative error: 0.3430778827615344
rank: 3


Biggest singular value = 2895.573280508755
Iter 29; Relative error: 0.34293950344012536
rank: 3


Biggest singular value = 2895.657920263877
Iter 30; Relative error: 0.3325953572978241
rank: 4


Biggest singular value = 2895.6056609808193
Iter 31; Relative error: 0.3264435896571843
rank: 4


Biggest singular value = 2895.4904377146986
Iter 32; Relative error: 0.3237515067756008
rank: 4


Biggest singular value = 2895.3765466508503
Iter 33; Relative error: 0.32252638729836053
rank: 4


Biggest singular value = 2895.2883729588834
Iter 34; Relative error: 0.3219236743392398
rank: 4


Biggest singular value = 2895.2311400601698
Iter 35; Relative error: 0.3170290082374964
rank: 5


Biggest singular value = 2895.1588645225934
Iter 36; Relative error: 0.3103975028904862
rank: 5


Biggest singular value = 2895.052523860267
Iter 37; Relative error: 0.3075674553756323
rank: 5


Biggest singular value = 2894.9642246274834
Iter 38; Relative err

Biggest singular value = 2882.532807884501
Iter 11; Relative error: 0.46973634905381373
rank: 2


Biggest singular value = 2888.903680218885
Iter 12; Relative error: 0.4278486869098297
rank: 2


Biggest singular value = 2891.9245734669976
Iter 13; Relative error: 0.4082804964911204
rank: 2


Biggest singular value = 2893.216723161257
Iter 14; Relative error: 0.3993025768525856
rank: 2


Biggest singular value = 2893.6654187701033
Iter 15; Relative error: 0.39512361742604296
rank: 2


Biggest singular value = 2893.737459792177
Iter 16; Relative error: 0.39308971116656893
rank: 2


Biggest singular value = 2893.6680740761453
Iter 17; Relative error: 0.39202361730461205
rank: 2


Biggest singular value = 2893.568097684788
Iter 18; Relative error: 0.3831437461656458
rank: 3


Biggest singular value = 2893.4132965836825
Iter 19; Relative error: 0.36145122125836743
rank: 3


Biggest singular value = 2893.102829797696
Iter 20; Relative error: 0.3521480912516813
rank: 3


Biggest singular valu

Biggest singular value = 2888.3523374761994
Iter 95; Relative error: 0.22714859477091812
rank: 18


Biggest singular value = 2888.229975570255
Iter 96; Relative error: 0.225271247705555
rank: 20


Biggest singular value = 2888.0643887001847
Iter 97; Relative error: 0.22320051907670085
rank: 20


Biggest singular value = 2887.852363783296
Iter 98; Relative error: 0.22197826614809651
rank: 21


Biggest singular value = 2887.661370536313
Iter 99; Relative error: 0.2200672171875587
rank: 22


Biggest singular value = 2887.4652372330265
Iter 100; Relative error: 0.21828969749461594
rank: 22


Biggest singular value = 2887.255547451653
Iter 101; Relative error: 0.215862232807074
rank: 24


Biggest singular value = 2886.9973435461065

 

At rank 25 and keep ratio 0.3:
Average General Error: 15.343780635106597%
Average In-Sample Error: 14.137276794659432%
Average Out-of-Sample Error: 15.83233584968579%
Average Sparsity: 0.0%
Average Time: 365.90 seconds
Average Iterations: 101.80




In [12]:
g_,is_,os_,t,k = average_general_error(max_rank=25, keep_ratio = 0.5,num_runs=5)







RUNNING 0






The biggest singular value on iteration 0: 1897.4854763034143
Biggest singular value = 1897.4854763034143
Iter 2; Relative error: 0.9227204333696878
rank: 1


Biggest singular value = 2465.0493942305557
Iter 3; Relative error: 0.5800866791925455
rank: 1


Biggest singular value = 2693.558220180361
Iter 4; Relative error: 0.5024729715201547
rank: 1


Biggest singular value = 2786.1736825223425
Iter 5; Relative error: 0.4879633886241954
rank: 1


Biggest singular value = 2824.1908132372796
Iter 6; Relative error: 0.4850463122264354
rank: 1


Biggest singular value = 2840.119494129185
Iter 7; Relative error: 0.4842481844946071
rank: 1


Biggest singular value = 2847.015783147411
Iter 8; Relative error: 0.445913160489133
rank: 2


Biggest singular value = 2848.6727194577397
Iter 9; Relative error: 0.40253460586795786
rank: 2


Biggest singular value = 2847.1390474974028
Iter 10; Relative error: 0.39340633560827265
rank: 2


Biggest singular value = 2845.615068532401
I

Iter 9; Relative error: 0.401472587024349
rank: 2


Biggest singular value = 2857.6360886603275
Iter 10; Relative error: 0.39237593577990093
rank: 2


Biggest singular value = 2856.074938207603
Iter 11; Relative error: 0.39029534363528035
rank: 2


Biggest singular value = 2855.066679510894
Iter 12; Relative error: 0.38964983689634847
rank: 2


Biggest singular value = 2854.5032081484737
Iter 13; Relative error: 0.35600237044527205
rank: 3


Biggest singular value = 2854.0281848743157
Iter 14; Relative error: 0.3460098125545567
rank: 3


Biggest singular value = 2853.749366484178
Iter 15; Relative error: 0.3441656793566888
rank: 3


Biggest singular value = 2853.6377816305753
Iter 16; Relative error: 0.3437031123608378
rank: 3


Biggest singular value = 2853.6166627919733
Iter 17; Relative error: 0.3434964845300904
rank: 3


Biggest singular value = 2853.6347779702523
Iter 18; Relative error: 0.3433549325443874
rank: 3


Biggest singular value = 2853.6659921981627
Iter 19; Relative err

Biggest singular value = 2852.719692101312
Iter 19; Relative error: 0.3440924672208814
rank: 3


Biggest singular value = 2852.7502918011915
Iter 20; Relative error: 0.3277474409187969
rank: 4


Biggest singular value = 2852.5636975267885
Iter 21; Relative error: 0.3233347414285419
rank: 4


Biggest singular value = 2852.391570681646
Iter 22; Relative error: 0.3224481026211674
rank: 4


Biggest singular value = 2852.290878751626
Iter 23; Relative error: 0.31684327145086355
rank: 5


Biggest singular value = 2852.2255248092506
Iter 24; Relative error: 0.3077708013121386
rank: 5


Biggest singular value = 2852.1718197704276
Iter 25; Relative error: 0.30617352903118783
rank: 5


Biggest singular value = 2852.1716263026296
Iter 26; Relative error: 0.29434827613857284
rank: 6


Biggest singular value = 2852.003607150766
Iter 27; Relative error: 0.28603034311627745
rank: 7


Biggest singular value = 2851.810341334703
Iter 28; Relative error: 0.2794357313307626
rank: 7


Biggest singular valu

Iter 28; Relative error: 0.2800179039107551
rank: 7


Biggest singular value = 2836.0899068476515
Iter 29; Relative error: 0.276806448232331
rank: 7


Biggest singular value = 2835.948406266122
Iter 30; Relative error: 0.276167694226208
rank: 7


Biggest singular value = 2835.889837259491
Iter 31; Relative error: 0.2759536533308941
rank: 7


Biggest singular value = 2835.885948535417
Iter 32; Relative error: 0.2754434396991288
rank: 8


Biggest singular value = 2835.9058923610337
Iter 33; Relative error: 0.26759704209172575
rank: 8


Biggest singular value = 2835.808722797663
Iter 34; Relative error: 0.2662540561323883
rank: 8


Biggest singular value = 2835.7526426543586
Iter 35; Relative error: 0.26594336600578444
rank: 8


Biggest singular value = 2835.7440303202466
Iter 36; Relative error: 0.26580376414175977
rank: 8


Biggest singular value = 2835.7646896718725
Iter 37; Relative error: 0.26569979830219503
rank: 8


Biggest singular value = 2835.799938488286
Iter 38; Relative error

Biggest singular value = 2849.2219402965616
Iter 36; Relative error: 0.2678664965587537
rank: 8


Biggest singular value = 2849.2459038346556
Iter 37; Relative error: 0.2677678071109907
rank: 8


Biggest singular value = 2849.2830592815358
Iter 38; Relative error: 0.267681378163331
rank: 8


Biggest singular value = 2849.3256219140644
Iter 39; Relative error: 0.26518511131715833
rank: 9


Biggest singular value = 2849.338588231099
Iter 40; Relative error: 0.26149788237309457
rank: 9


Biggest singular value = 2849.2993372606124
Iter 41; Relative error: 0.2608306371022919
rank: 9


Biggest singular value = 2849.2852436100397
Iter 42; Relative error: 0.2606564236649613
rank: 9


Biggest singular value = 2849.2971410599534
Iter 43; Relative error: 0.26056742766736657
rank: 9


Biggest singular value = 2849.324404750535
Iter 44; Relative error: 0.259924551467646
rank: 10


Biggest singular value = 2849.351300797966
Iter 45; Relative error: 0.2558498008330519
rank: 10


Biggest singular val

In [13]:
g_,is_,os_,t,k = average_general_error(max_rank=50, keep_ratio = 0.3,num_runs=5)







RUNNING 0






The biggest singular value on iteration 0: 1999.5995112850887
Biggest singular value = 1999.5995112850887
Iter 2; Relative error: 0.8584123874034284
rank: 1


Biggest singular value = 2323.29978953552
Iter 3; Relative error: 0.6637688020561275
rank: 1


Biggest singular value = 2530.0647952189615
Iter 4; Relative error: 0.5651397110972929
rank: 1


Biggest singular value = 2662.4234163007254
Iter 5; Relative error: 0.5189507083078332
rank: 1


Biggest singular value = 2747.4198005235103
Iter 6; Relative error: 0.4982674678237141
rank: 1


Biggest singular value = 2802.2174208182423
Iter 7; Relative error: 0.48906927717190246
rank: 1


Biggest singular value = 2837.7131467774775
Iter 8; Relative error: 0.4848748842145983
rank: 1


Biggest singular value = 2860.835505979652
Iter 9; Relative error: 0.4828602548981538
rank: 1


Biggest singular value = 2875.998947493507
Iter 10; Relative error: 0.4818185509921168
rank: 1


Biggest singular value = 2886.022924280799
It

Biggest singular value = 2893.1361367224695
Iter 85; Relative error: 0.23894033018440905
rank: 14


Biggest singular value = 2892.98427337856
Iter 86; Relative error: 0.2378115064264435
rank: 14


Biggest singular value = 2892.8397074438326
Iter 87; Relative error: 0.23550181710960763
rank: 15


Biggest singular value = 2892.6680403901023
Iter 88; Relative error: 0.23448862553893413
rank: 15


Biggest singular value = 2892.521472527531
Iter 89; Relative error: 0.23392265859284195
rank: 16


Biggest singular value = 2892.4129143714317
Iter 90; Relative error: 0.2321327735619851
rank: 16


Biggest singular value = 2892.2714242531374
Iter 91; Relative error: 0.23142605863227309
rank: 16


Biggest singular value = 2892.153942732639
Iter 92; Relative error: 0.23003153986734476
rank: 17


Biggest singular value = 2892.0137551695257
Iter 93; Relative error: 0.22914639635011194
rank: 17


Biggest singular value = 2891.8797190857613
Iter 94; Relative error: 0.22766892355157875
rank: 19


Bigges

Biggest singular value = 2894.766278622123
Iter 39; Relative error: 0.29575389826103565
rank: 6


Biggest singular value = 2894.5543244206765
Iter 40; Relative error: 0.2930855279659542
rank: 6


Biggest singular value = 2894.359440621753
Iter 41; Relative error: 0.28725607436884754
rank: 7


Biggest singular value = 2894.1225726518346
Iter 42; Relative error: 0.2819389955513057
rank: 7


Biggest singular value = 2893.8511638575906
Iter 43; Relative error: 0.2796625064365366
rank: 7


Biggest singular value = 2893.62024805135
Iter 44; Relative error: 0.27862413159413635
rank: 7


Biggest singular value = 2893.4524466463913
Iter 45; Relative error: 0.27809152261423287
rank: 7


Biggest singular value = 2893.346855016876
Iter 46; Relative error: 0.2777702285212019
rank: 7


Biggest singular value = 2893.2936781242574
Iter 47; Relative error: 0.277541611690057
rank: 7


Biggest singular value = 2893.2811662659633
Iter 48; Relative error: 0.27516440306055095
rank: 8


Biggest singular valu

Biggest singular value = 2885.8186904317363
Iter 122; Relative error: 0.18304380139160634
rank: 45


Biggest singular value = 2885.650069972797
Iter 123; Relative error: 0.18147806302253788
rank: 45


Biggest singular value = 2885.473355615847
Iter 124; Relative error: 0.18045261100874327
rank: 46


Biggest singular value = 2885.3197607729626
Iter 125; Relative error: 0.17934833773679537
rank: 47


Biggest singular value = 2885.1728943355547
Iter 126; Relative error: 0.17830910364819783
rank: 47


Biggest singular value = 2885.041322121242
Iter 127; Relative error: 0.17764483653198718
rank: 47


Biggest singular value = 2884.941248936639
Iter 128; Relative error: 0.17680068698929366
rank: 48


Biggest singular value = 2884.853627325524
Iter 129; Relative error: 0.17565414756450334
rank: 49


Biggest singular value = 2884.75683874702
Iter 130; Relative error: 0.17415313516733022
rank: 50


Biggest singular value = 2884.6338998723777






RUNNING 2






The biggest singular value on it

Iter 75; Relative error: 0.252261870093715
rank: 10


Biggest singular value = 2894.0391908253478
Iter 76; Relative error: 0.251678294617318
rank: 11


Biggest singular value = 2894.0727445714024
Iter 77; Relative error: 0.24884582690908627
rank: 12


Biggest singular value = 2894.0167801172156
Iter 78; Relative error: 0.24600799483896196
rank: 12


Biggest singular value = 2893.8713682016414
Iter 79; Relative error: 0.24482533971618053
rank: 12


Biggest singular value = 2893.7223377219552
Iter 80; Relative error: 0.244293422777615
rank: 12


Biggest singular value = 2893.602890964255
Iter 81; Relative error: 0.24401932082610225
rank: 12


Biggest singular value = 2893.5212220382646
Iter 82; Relative error: 0.24384968629981316
rank: 12


Biggest singular value = 2893.4748368765127
Iter 83; Relative error: 0.2423243468939415
rank: 13


Biggest singular value = 2893.4021724825916
Iter 84; Relative error: 0.24038186533276568
rank: 14


Biggest singular value = 2893.287676831995
Iter 85; 

Biggest singular value = 2897.2067806321324
Iter 28; Relative error: 0.3444257746801199
rank: 3


Biggest singular value = 2897.287776283773
Iter 29; Relative error: 0.3413536759107297
rank: 4


Biggest singular value = 2897.334059313321
Iter 30; Relative error: 0.3308887036988589
rank: 4


Biggest singular value = 2897.234759377939
Iter 31; Relative error: 0.32633684851448513
rank: 4


Biggest singular value = 2897.1022932639726
Iter 32; Relative error: 0.32431873050737087
rank: 4


Biggest singular value = 2896.985065121008
Iter 33; Relative error: 0.32337557591255417
rank: 4


Biggest singular value = 2896.8992791781106
Iter 34; Relative error: 0.3228914408473004
rank: 4


Biggest singular value = 2896.846187779562
Iter 35; Relative error: 0.3165503662151834
rank: 5


Biggest singular value = 2896.7617213886715
Iter 36; Relative error: 0.3107981362591121
rank: 5


Biggest singular value = 2896.6548258307125
Iter 37; Relative error: 0.30833956024552234
rank: 5


Biggest singular valu

Biggest singular value = 2889.3170567349875
Iter 111; Relative error: 0.19946098697552792
rank: 34


Biggest singular value = 2889.15900809599
Iter 112; Relative error: 0.19803233824950814
rank: 35


Biggest singular value = 2889.001962937157
Iter 113; Relative error: 0.19660372209213114
rank: 35


Biggest singular value = 2888.8434721896015
Iter 114; Relative error: 0.1957885524004034
rank: 36


Biggest singular value = 2888.715464191065
Iter 115; Relative error: 0.19450589257242445
rank: 36


Biggest singular value = 2888.583824135794
Iter 116; Relative error: 0.19303884462275542
rank: 38


Biggest singular value = 2888.4420743365936
Iter 117; Relative error: 0.19157947326716987
rank: 39


Biggest singular value = 2888.2885815085297
Iter 118; Relative error: 0.19001401873735785
rank: 40


Biggest singular value = 2888.1257158331946
Iter 119; Relative error: 0.18812757632108487
rank: 42


Biggest singular value = 2887.9484075797936
Iter 120; Relative error: 0.186245084854459
rank: 42


Iter 64; Relative error: 0.2597939918649742
rank: 9


Biggest singular value = 2883.3266674917204
Iter 65; Relative error: 0.2596736155914832
rank: 9


Biggest singular value = 2883.3706005575186
Iter 66; Relative error: 0.2595666465383234
rank: 9


Biggest singular value = 2883.4217628275746
Iter 67; Relative error: 0.25646852554913463
rank: 10


Biggest singular value = 2883.4134649988086
Iter 68; Relative error: 0.2549435370009177
rank: 10


Biggest singular value = 2883.3874465678173
Iter 69; Relative error: 0.2542594971129524
rank: 10


Biggest singular value = 2883.3676292302825
Iter 70; Relative error: 0.25392038427943625
rank: 10


Biggest singular value = 2883.362254441837
Iter 71; Relative error: 0.2537244468201444
rank: 10


Biggest singular value = 2883.3722868072964
Iter 72; Relative error: 0.2535896759427398
rank: 10


Biggest singular value = 2883.3957325893025
Iter 73; Relative error: 0.25348257255595497
rank: 10


Biggest singular value = 2883.429740053247
Iter 74; Rel

In [14]:
g_,is_,os_,t,k = average_general_error(max_rank=50, keep_ratio = 0.5,num_runs=5)







RUNNING 0






The biggest singular value on iteration 0: 1917.0069677436184
Biggest singular value = 1917.0069677436184
Iter 2; Relative error: 0.9099262892282932
rank: 1


Biggest singular value = 2479.390605817634
Iter 3; Relative error: 0.5765270107801015
rank: 1


Biggest singular value = 2705.7507094538955
Iter 4; Relative error: 0.5015258945007354
rank: 1


Biggest singular value = 2797.4906480528675
Iter 5; Relative error: 0.48752094059012446
rank: 1


Biggest singular value = 2835.15681340561
Iter 6; Relative error: 0.4846920211601909
rank: 1


Biggest singular value = 2850.9476461767167
Iter 7; Relative error: 0.48391142251217945
rank: 1


Biggest singular value = 2857.7913959171765
Iter 8; Relative error: 0.44465315283445006
rank: 2


Biggest singular value = 2859.332308928517
Iter 9; Relative error: 0.403223514753246
rank: 2


Biggest singular value = 2857.7448227251407
Iter 10; Relative error: 0.3944802222306191
rank: 2


Biggest singular value = 2856.192678501745
I

Biggest singular value = 2850.307260782219
Iter 85; Relative error: 0.19737171811515195
rank: 35


Biggest singular value = 2850.18839051981
Iter 86; Relative error: 0.19503557252952045
rank: 37


Biggest singular value = 2850.065925307817
Iter 87; Relative error: 0.19263305722249857
rank: 38


Biggest singular value = 2849.9332975839743
Iter 88; Relative error: 0.19107327639194238
rank: 39


Biggest singular value = 2849.8304577442836
Iter 89; Relative error: 0.18888293524518499
rank: 41


Biggest singular value = 2849.719116645408
Iter 90; Relative error: 0.187393060663978
rank: 42


Biggest singular value = 2849.6256650491405
Iter 91; Relative error: 0.1857300055411827
rank: 43


Biggest singular value = 2849.544228544301
Iter 92; Relative error: 0.18329240739797828
rank: 45


Biggest singular value = 2849.434395998578
Iter 93; Relative error: 0.18222298554590882
rank: 46


Biggest singular value = 2849.3635988566816
Iter 94; Relative error: 0.1800703285859713
rank: 48


Biggest sin

Iter 72; Relative error: 0.22129784983199077
rank: 21


Biggest singular value = 2840.6812140281927
Iter 73; Relative error: 0.2196677454162193
rank: 22


Biggest singular value = 2840.612918725459
Iter 74; Relative error: 0.21723656089500215
rank: 23


Biggest singular value = 2840.479933612119
Iter 75; Relative error: 0.2151071928622988
rank: 25


Biggest singular value = 2840.3451498707395
Iter 76; Relative error: 0.21229530895557108
rank: 26


Biggest singular value = 2840.1820541250545
Iter 77; Relative error: 0.21130434687089164
rank: 26


Biggest singular value = 2840.0778703653577
Iter 78; Relative error: 0.21102013574029527
rank: 26


Biggest singular value = 2840.0398280035897
Iter 79; Relative error: 0.20964502761154516
rank: 27


Biggest singular value = 2839.9869954674264
Iter 80; Relative error: 0.2081175831009567
rank: 28


Biggest singular value = 2839.9228496463766
Iter 81; Relative error: 0.20563159890071103
rank: 30


Biggest singular value = 2839.816610271235
Iter 8

Biggest singular value = 2844.047799520747
Iter 57; Relative error: 0.245651432769893
rank: 13


Biggest singular value = 2844.042828364007
Iter 58; Relative error: 0.24250753725625337
rank: 14


Biggest singular value = 2843.9779110354057
Iter 59; Relative error: 0.23956785660466456
rank: 14


Biggest singular value = 2843.886569771092
Iter 60; Relative error: 0.23802555911341303
rank: 15


Biggest singular value = 2843.8119652948053
Iter 61; Relative error: 0.23628226868848798
rank: 15


Biggest singular value = 2843.72300637185
Iter 62; Relative error: 0.23598241162451228
rank: 15


Biggest singular value = 2843.6837483797194
Iter 63; Relative error: 0.23587202700147514
rank: 15


Biggest singular value = 2843.682118421843
Iter 64; Relative error: 0.23579029375748228
rank: 15


Biggest singular value = 2843.701393245476
Iter 65; Relative error: 0.2325663123887668
rank: 17


Biggest singular value = 2843.6229523473025
Iter 66; Relative error: 0.22936687971867908
rank: 18


Biggest si

Iter 43; Relative error: 0.2597274836613736
rank: 9


Biggest singular value = 2849.5255433284037
Iter 44; Relative error: 0.25805759259484295
rank: 10


Biggest singular value = 2849.540849251026
Iter 45; Relative error: 0.25478343299842166
rank: 10


Biggest singular value = 2849.5090797741063
Iter 46; Relative error: 0.25418416159135704
rank: 10


Biggest singular value = 2849.497453366182
Iter 47; Relative error: 0.2540280236148338
rank: 10


Biggest singular value = 2849.5080162981303
Iter 48; Relative error: 0.25394922207313403
rank: 10


Biggest singular value = 2849.5322428397335
Iter 49; Relative error: 0.2538873833045623
rank: 10


Biggest singular value = 2849.563253554353
Iter 50; Relative error: 0.25383203024936424
rank: 10


Biggest singular value = 2849.5970209023753
Iter 51; Relative error: 0.2537809847278267
rank: 10


Biggest singular value = 2849.631474371354
Iter 52; Relative error: 0.25065507775829315
rank: 11


Biggest singular value = 2849.582600496542
Iter 53; R

Iter 30; Relative error: 0.27621849234109336
rank: 7


Biggest singular value = 2845.6622324941745
Iter 31; Relative error: 0.2760458224951169
rank: 7


Biggest singular value = 2845.671172826379
Iter 32; Relative error: 0.26934551637556264
rank: 8


Biggest singular value = 2845.600134290971
Iter 33; Relative error: 0.2664969300086283
rank: 8


Biggest singular value = 2845.5249555365353
Iter 34; Relative error: 0.2659446077897079
rank: 8


Biggest singular value = 2845.4997496579144
Iter 35; Relative error: 0.2657593465173489
rank: 8


Biggest singular value = 2845.5121046459517
Iter 36; Relative error: 0.26564105126731613
rank: 8


Biggest singular value = 2845.5448558120993
Iter 37; Relative error: 0.2655416445764281
rank: 8


Biggest singular value = 2845.5868464441096
Iter 38; Relative error: 0.265452379234605
rank: 8


Biggest singular value = 2845.6321464377943
Iter 39; Relative error: 0.2649499338489296
rank: 9


Biggest singular value = 2845.673189005259
Iter 40; Relative err